In [130]:
import pandas as pd 
import numpy as np 
from zipfile import ZipFile
import os
import glob
import plotly.express as px

In [88]:
path = os.getcwd()
print(path)
print(os.listdir("prem/English Premier League (football) 10 Seasons/"))

C:\Users\mgxsl\github\PythonWork
['season-0910_csv.csv', 'season-1011_csv.csv', 'season-1112_csv.csv', 'season-1213_csv.csv', 'season-1314_csv.csv', 'season-1415_csv.csv', 'season-1516_csv.csv', 'season-1617_csv.csv', 'season-1718_csv.csv', 'season-1819_csv.csv']


In [131]:
glob.glob("*.zip")

['prem.zip']

In [111]:
### Read in from unzipped folder "prem"

file_list = list() 
path_to_csv = "prem/English Premier League (football) 10 Seasons/"
for file in os.listdir(path_to_csv):
#     print(path_to_csv + file)
    df=pd.read_csv(path_to_csv + file,sep=",")
    df['filename'] = file
    file_list.append(df)
    
all_szn = pd.concat(file_list, axis=0, ignore_index=True)
        

prem/English Premier League (football) 10 Seasons/season-0910_csv.csv
season-0910_csv.csv
prem/English Premier League (football) 10 Seasons/season-1011_csv.csv
season-1011_csv.csv
prem/English Premier League (football) 10 Seasons/season-1112_csv.csv
season-1112_csv.csv
prem/English Premier League (football) 10 Seasons/season-1213_csv.csv
season-1213_csv.csv
prem/English Premier League (football) 10 Seasons/season-1314_csv.csv
season-1314_csv.csv
prem/English Premier League (football) 10 Seasons/season-1415_csv.csv
season-1415_csv.csv
prem/English Premier League (football) 10 Seasons/season-1516_csv.csv
season-1516_csv.csv
prem/English Premier League (football) 10 Seasons/season-1617_csv.csv
season-1617_csv.csv
prem/English Premier League (football) 10 Seasons/season-1718_csv.csv
season-1718_csv.csv
prem/English Premier League (football) 10 Seasons/season-1819_csv.csv
season-1819_csv.csv


In [142]:
all_szn.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,filename,PSH,PSD,PSA,PSCH,PSCD,PSCA
0,E0,2009-08-15,Aston Villa,Wigan,0,2,A,0,1,A,...,1.22,4.40,3.99,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2009-08-15,Blackburn,Man City,0,2,A,0,1,A,...,2.38,1.60,1.54,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2009-08-15,Bolton,Sunderland,0,1,A,0,1,A,...,1.61,2.33,2.23,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2009-08-15,Chelsea,Hull,2,1,H,1,1,D,...,1.02,17.05,12.96,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2009-08-15,Everton,Arsenal,1,6,A,0,3,A,...,2.20,1.73,1.63,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN


In [245]:
## est. empty df
df_master = pd.DataFrame()

zip_files = glob.glob("*.zip")
#load all zip files in folder

## load the zip file first  - only one is read but i theory we could loop through several .zip
for zipfilename in zip_files:
#     print(zipfilename) 
    ## extract zip with ZipFile()
    zip_file = ZipFile(zipfilename)
    ## ZipFile.infolist() function is useful as it returns a list of the files zipped in the zipfolder, incl \
    ## individually call 
#     print(zip_file.infolist())
    
    for csv in zip_file.infolist():
        ## print full info of zip folder file
#         print(csv)
        ## print only the name 
#         print(csv.filename)
        ## read in the specific csv file in the zip folder. not wrap pandas read_csv around the ZipFile.open() function
        df = pd.read_csv(zip_file.open(csv.filename), delimiter=',',header=0)
        ## add col with specific file identifier 
        csv_name = str.split(csv.filename,"/")[1]
#         print(csv_name)
        df['filename'] = csv_name 
        ## concat the dfs from the zip folder
        df_master = pd.concat([df_master, df])

# print(df_master.info())

In [ ]:
df_master.sample(10)

In [177]:
display(df_master.dtypes.head(20),len(df_master),len(df_master.columns))

Div         object
Date        object
HomeTeam    object
AwayTeam    object
FTHG         int64
FTAG         int64
FTR         object
HTHG         int64
HTAG         int64
HTR         object
Referee     object
HS           int64
AS           int64
HST          int64
AST          int64
HF           int64
AF           int64
HC           int64
AC           int64
HY           int64
dtype: object

3800

78

In [234]:
mancity_home.sample(5)

print("number of matches:", len(df_master[df_master.HomeTeam == "Everton"]))

number of man city games: 190


In [239]:
print("number of man city games:", len(df_master[df_master.HomeTeam == "Man City"]))

mancity_home = df_master[df_master.HomeTeam == "Man City"]
      
print(mancity_home["FTHG"].dtypes, mancity_home["FTAG"].dtypes)

mancity_home["Goal Diff"] = mancity_home["FTHG"] - mancity_home["FTAG"]

print("number of man city games w/ goal diff:", len(mancity_home["Goal Diff"]))
      
gp = mancity_home.groupby('Goal Diff').count()

display(gp.sample(4))

fig = px.bar(x = gp.index, y = gp.Date, color = gp.index,
             color_continuous_scale=px.colors.sequential.Cividis,
             labels = dict(y ="# of Games",
                           x ="Goal Difference/Game")
            )

fig.show()

## note - date aren't all the same 

number of man city games: 190
int64 int64
number of man city games w/ goal diff: 190


C:\Users\mgxsl\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,filename,PSH,PSD,PSA,PSCH,PSCD,PSCA
Goal Diff,,,,,,,,,,,,,,,,,,,,,
1,47,47,47,47,47,47,47,47,47,47,...,47,47,47,47,31,31,31,31,31,31
2,39,39,39,39,39,39,39,39,39,39,...,39,39,39,39,29,29,29,29,29,29
6,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
-2,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,3,3,3,3,3,3


In [244]:

print(mancity_home.Date.head(5), mancity_home.Date.tail(5))
print('dates need fixing here')


19     2009-08-22
38     2009-09-12
65     2009-09-28
94     2009-10-25
109    2009-11-07
Name: Date, dtype: object 277    27/02/2019
293    09/03/2019
317    03/04/2019
338    20/04/2019
369    06/05/2019
Name: Date, dtype: object
dates need fixing here


### BeautifulSoup


In [146]:
import requests
from bs4 import BeautifulSoup 


200

In [147]:
response = requests.get("")
response.status_code



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="this page displays the current week of the year" name="description"/>
<meta content="date, week, calculator" name="keywords"/>
<title>What week of the year is it?</title>
<link href="css/sunny/jquery-ui-1.7.2.custom.css" rel="stylesheet" type="text/css"/>
<link href="css/screen.css" media="screen, projection" rel="stylesheet" type="text/css"/>
<link href="css/print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="css/plugins/fancy-type/screen.css" media="screen, projection" rel="stylesheet" type="text/css"/>
<link href="favicon.ico" rel="shortcut icon"/>
<script src="js/jquery-1.3.2.min.js" type="text/javascript"></script>
<script src="js/jquery-ui-1.7.2.custom.min.js" type="text/javascript"></script>
<script src="js/functions.js" type="text/javascript"></script>
<script src="//connect.facebook.net/en_US/all

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")
soup

In [151]:
print(soup.title)
title = soup.title
print(title.string)

## find all p tags, that is paragraph
soup.find_all("p")


<title>What week of the year is it?</title>
What week of the year is it?


[<p>This single serving app calculates the week of the year and day of the year for the current day and for any day which you specify. Select a date from the calendar to see this in action.</p>,
 <p>Please note that this site uses the ISO week date system. This is used primarily for financial and government timekeeping. For more information, please refer to this <a href="https://en.wikipedia.org/wiki/ISO_week_date" target="_blank">Wikipedia article</a>.</p>]

In [159]:
cal_table = soup.find("table")
cal_table_info = cal_table.find_all("td")
print(cal_table_info)
for date in cal_table_info:
    print(date.get_text())
    

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

The only difference is that `find_all()` returns a list containing the single result, and `find()` just returns the result.

If `find_all()` can’t find anything, it returns an empty list. If `find()` can’t find anything, it returns None: